In [50]:
import lsst.sims.maf.metrics.baseMetric
import numpy as np
import random
from lsst.sims.maf.metrics import BaseMetric, ExgalM5
from lsst.sims.maf.maps import DustMap
from lsst.sims.maf.slicers import HealpixSlicer
import lsst.sims.maf.stackers as stackers
import lsst.sims.maf.db as db
import lsst.sims.maf.metricBundles as metricBundles
import pickle


In [51]:
import warnings
warnings.filterwarnings('ignore')
%pylab inline
from scipy import stats
from angles import r2d, r2arcs, d2arcs, arcs2r
def arcm2r(theta):
    return arcs2r(theta*60)
import seaborn as sns;sns.set_style('darkgrid')
import lsst.sims.maf.stackers as stackers
import treecorr as tr
%config InlineBackend.figure_format = 'retina'
matplotlib.rcParams['figure.figsize'] = 19,14
import lsst.sims.maf.slicers as slicers
import lsst.sims.maf.metrics as metrics
import lsst.sims.maf.metricBundles as metricBundles
import lsst.sims.maf.db as db
from collections import defaultdict
import lsst.sims.maf.maps as maps
import treecorr
from itertools import chain
sns.set_context('poster',font_scale=1.4)
import random
import lsst.sims.maf.batches as batches
from itertools import groupby
from weakLensingSystematicsMetricCopy1 import AverageVisitsMetric

Populating the interactive namespace from numpy and matplotlib


In [52]:
newestSimsPath = '/global/cscratch1/sd/awan/dbs_post_wp_v2/'
newestSims = !ls /global/cscratch1/sd/awan/dbs_post_wp_v2/
newestSims = [sim[:-3] for sim in newestSims]

In [53]:
d = {}
nside = 256
outDir = 'temp'
resultsDb = db.ResultsDb(outDir=outDir)

In [5]:
dbFile = newestSimsPath+newestSims[0]+'.db'
conn = db.OpsimDatabase(dbFile)

In [54]:
import healpy as hp

In [7]:
class WackySummaryMetric(metrics.BaseMetric):
    def __init__(self, nside, col=None, **kwargs):
        self.nside = nside
        super(WackySummaryMetric, self).__init__(metricName='wacky',col=col, **kwargs)
        if col is None:
            self.col = 'metricdata'
    def run(self, dataSlice, slicePoint=None):
        # let's assume we are running on a coadded depth
        
        # Chop off any outliers
        print(self.col)
        
        good_pix = np.where(np.logical_and(dataSlice[self.col] > 26, slicePoint['ebv'] < 0.02))[0]
        print(dataSlice)
        # area = hp.nside2pixarea(self.nside, degrees=True) * np.size(good_pix)
        area = len(good_pix)*hp.pixelfunc.nside2pixarea(nside=nside, degrees=True)
        median_depth = np.median(dataSlice[self.col][good_pix])
        # Here's some value with strange units of mags*deg^2
        return area

In [8]:
nside = 256
bundleList = []
sql = 'filter = "i" and note not like "DD%"'
dustMap = maps.DustMap(interp=False, nside=nside)

metric = metrics.ExgalM5(maps=['DustMap'])
slicer = slicers.HealpixSlicer(nside=nside)
summary = [WackySummaryMetric(nside)]
summary.append(metrics.MedianMetric())
bundleList.append(metricBundles.MetricBundle(metric,slicer,sql, summaryMetrics=summary))

bd = metricBundles.makeBundlesDictFromList(bundleList)
bg = metricBundles.MetricBundleGroup(bd, conn, outDir=outDir, resultsDb=resultsDb)
bg.runAll()


bundleList[0].summaryValues


Healpix slicer using NSIDE=256, approximate resolution 13.741946 arcminutes
Querying database SummaryAllProps with constraint filter = "i" and note not like "DD%" for columns ['fieldRA', 'fiveSigmaDepth', 'fieldDec']
Found 428965 visits
Running:  ['opsim_ExgalM5_fiveSigmaDepth_i_and_note_not_like_DD%_HEAL']
Completed metric generation.
Running reduce methods.
Running summary statistics.
metricdata


TypeError: 'NoneType' object is not subscriptable

In [ ]:


## Getting Area and Median Depth
year = 1
sqlWhere = 'night < '+str(year*365+1)+' and filter = "i" and note not like "DD%"'

slicer = slicers.HealpixSlicer(
    lonCol='fieldRA', latCol='fieldDec', nside=nside, useCache=False
    )

metric = metrics.ExgalM5(lsstFilter='i')
dustMap = maps.DustMap(interp=False, nside=nside)
myBundles['0'] = metricBundles.MetricBundle(metric, slicer, constraint=sqlWhere, 
                                                       stackerList=stackerList, runName=runName,
                                                       metadata='fom_prep', mapsList=[dustMap])
bgroup = metricBundles.MetricBundleGroup(myBundles, opsdb, outDir=outDir, resultsDb=resultsDb)
bgroup.runAll() 
bundle = myBundles['0']


depth_cut = {1:24.5, 3:25, 6:25.5, 10:26}
cond = np.logical_and.reduce((bundle.slicer.getSlicePoints()['ebv']<0.2, bundle.metricValues.mask==False, bundle.metricValues.data>depth_cit))
depths = bundle.metricValues[cond]
median_depth = np.median(depths)
area = len(bundle.metricValues[cond])*hp.pixelfunc.nside2pixarea(nside=nside, degrees=True)





In [18]:
## Calculating FoM
if year == 1:
    areas = [7500, 13000, 16000]
    depths = [24.9, 25.2, 25.5]
    fom_arr = [
        [1.212257e+02, 1.462689e+02, 1.744913e+02],
        [1.930906e+02, 2.365094e+02, 2.849131e+02],
        [2.316956e+02, 2.851547e+02, 3.445717e+02]
    ]
elif year == 3:
    areas = [10000, 15000, 20000]
    depths = [25.5, 25.8, 26.1]
    fom_arr = [
        [1.710645e+02, 2.246047e+02, 2.431472e+02],
        [2.445209e+02, 3.250737e+02, 3.516395e+02],
        [3.173144e+02, 4.249317e+02, 4.595133e+02]
    ]

elif year == 6:
    areas = [10000, 15000, 2000]
    depths = [25.9, 26.1, 26.3]
    fom_arr = [
        [2.346060e+02, 2.414678e+02, 2.852043e+02],
        [3.402318e+02, 3.493120e+02, 4.148814e+02],
        [4.452766e+02, 4.565497e+02, 5.436992e+02]
    ]
    
elif year == 10:
    areas = [10000, 15000, 20000]
    depths = [26.3, 26.5, 26.7]
    fom_arr = [
        [2.887266e+02, 2.953230e+02, 3.361616e+02],
        [4.200093e+02, 4.292111e+02, 4.905306e+02],
        [5.504419e+02, 5.624697e+02, 6.441837e+02]
    ]


areas = [[i]*3 for i in areas]
depths = [list(depths)]*3
from scipy import interpolate
f = interpolate.interp2d(areas1, depths1, fom_arr, bounds_error=False)
fom = f([area[0] for area in areas1], depths1[0])
return fom

In [76]:
def fom(dbFile, year):
    print(dbFile)
    runName = newestSimsPath+dbFile+'.db'
    opsdb = db.OpsimDatabase(runName)
    if year == 10:
        sqlWhere = 'filter = "i" and note not like "DD%"'
    else:
        sqlWhere = 'night < '+str(year*365+1)+' and filter = "i" and note not like "DD%"'

    slicer = slicers.HealpixSlicer(
        lonCol='fieldRA', latCol='fieldDec', nside=nside, useCache=False
        )
    myBundles = {}
    metric = metrics.ExgalM5(lsstFilter='i')
    dustMap = maps.DustMap(interp=False, nside=nside)
    myBundles['0'] = metricBundles.MetricBundle(metric, slicer, constraint=sqlWhere, 
                                                           stackerList=[], runName=runName,
                                                           metadata='fom_prep', mapsList=[dustMap])
    bgroup = metricBundles.MetricBundleGroup(myBundles, opsdb, outDir=outDir, resultsDb=resultsDb)
    bgroup.runAll() 
    bundle = myBundles['0']


    depth_cut = {1:24.5, 3:25, 6:25.5, 10:26}
    cond = np.logical_and.reduce((bundle.slicer.getSlicePoints()['ebv']<0.2, bundle.metricValues.mask==False, bundle.metricValues.data>depth_cut[year]))
    median_depth = np.median(bundle.metricValues[cond])
    area = len(bundle.metricValues[cond])*hp.pixelfunc.nside2pixarea(nside=nside, degrees=True)
    if year == 1:
        areas = [7500, 13000, 16000]
        depths = [24.9, 25.2, 25.5]
        fom_arr = [
            [1.212257e+02, 1.462689e+02, 1.744913e+02],
            [1.930906e+02, 2.365094e+02, 2.849131e+02],
            [2.316956e+02, 2.851547e+02, 3.445717e+02]
        ]
    elif year == 3:
        areas = [10000, 15000, 20000]
        depths = [25.5, 25.8, 26.1]
        fom_arr = [
            [1.710645e+02, 2.246047e+02, 2.431472e+02],
            [2.445209e+02, 3.250737e+02, 3.516395e+02],
            [3.173144e+02, 4.249317e+02, 4.595133e+02]
        ]

    elif year == 6:
        areas = [10000, 15000, 2000]
        depths = [25.9, 26.1, 26.3]
        fom_arr = [
            [2.346060e+02, 2.414678e+02, 2.852043e+02],
            [3.402318e+02, 3.493120e+02, 4.148814e+02],
            [4.452766e+02, 4.565497e+02, 5.436992e+02]
        ]

    elif year == 10:
        areas = [10000, 15000, 20000]
        depths = [26.3, 26.5, 26.7]
        fom_arr = [
            [2.887266e+02, 2.953230e+02, 3.361616e+02],
            [4.200093e+02, 4.292111e+02, 4.905306e+02],
            [5.504419e+02, 5.624697e+02, 6.441837e+02]
        ]


    areas = [[i]*3 for i in areas]
    depths = [list(depths)]*3
    from scipy import interpolate
    f = interpolate.interp2d(areas, depths, fom_arr, bounds_error=False)
    fom = f(area, median_depth)[0]
    print(fom)
    return fom



In [77]:
import pickle
fomresults = {}

In [ ]:
for sim in newestSims:
    fomresults[sim] = fom(sim, 10)
    pickle.dump(fomresults, open('fom_results_y10.p', 'wb'))

add_mag_clouds_v1.3_10yrs
Healpix slicer using NSIDE=256, approximate resolution 13.741946 arcminutes
Querying database SummaryAllProps with constraint filter = "i" and note not like "DD%" for columns ['fieldRA', 'fiveSigmaDepth', 'fieldDec']
Found 428965 visits
Running:  ['0']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
435.637945126161
agnddf_illum10_v1.3_10yrs
Healpix slicer using NSIDE=256, approximate resolution 13.741946 arcminutes
Querying database SummaryAllProps with constraint filter = "i" and note not like "DD%" for columns ['fieldRA', 'fiveSigmaDepth', 'fieldDec']
Found 451194 visits
Running:  ['0']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
435.1238531143017
agnddf_illum15_v1.3_10yrs
Healpix slicer using NSIDE=256, approximate resolution 13.741946 arcminutes
Querying database SummaryAllProps with constraint filter = "i" and note not like "DD%" for columns ['fieldRA', 'fiveSigma

In [42]:
fom_arr

[[121.2257, 146.2689, 174.4913],
 [193.0906, 236.5094, 284.9131],
 [231.6956, 285.1547, 344.5717]]

In [43]:
renorm_value = -100.0



In [45]:
emulated_grid_fom_vals.T

array([[121.2257, 146.2689, 174.4913],
       [193.0906, 236.5094, 284.9131],
       [231.6956, 285.1547, 344.5717]])

In [46]:
f(14075.69, 25.06)

array([232.12693118])

In [47]:
f(15544.35, 24.90)

array([225.86034513])

In [48]:
f(15000, 25.06)

array([245.61477842])

In [39]:
[area[0] for area in areas1], depths1[0]

([7500.0, 13000.0, 16000.0], [24.9, 25.2, 25.5])